In [43]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd


region = "us-east-1" 
boto_session = boto3.Session(region_name=region)
sess = sagemaker.Session(boto_session=boto_session)
sm_boto3 = boto3.client("sagemaker", region_name=region)
bucket = "mobbucketsagemaker0210"
print("Using bucket " + bucket)



[01/29/25 19:27:36] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=511358;file://d:\MLOps\awssagemaker\venv\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=766013;file://d:\MLOps\awssagemaker\venv\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

Using bucket mobbucketsagemaker0210


In [44]:
df=pd.read_csv("mpc_train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1


In [45]:
df.shape

(2000, 21)

In [46]:
df.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [47]:
df['price_range'].value_counts()


price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [48]:
features=list(df.columns)
features


['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [49]:
lable=features.pop(-1)
lable

'price_range'

In [50]:
x=df[features]
y=df[lable]

In [51]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y, test_size=0.15, random_state=0) 
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)


(1700, 20)
(300, 20)
(1700,)
(300,)


In [52]:
trainX = pd.DataFrame(X_train)
trainX[lable] = Y_train

testX = pd.DataFrame(X_test)
testX[lable] = Y_test

In [53]:
trainX

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,4,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,14,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,13,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,4,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,20,48,1012,959,17,7,6,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1224,1,1.6,0,9,0,33,1.0,157,1,18,522,563,3796,10,5,13,1,1,0,3
1216,1158,0,0.7,1,1,1,29,0.7,123,2,20,311,1796,1542,17,9,15,1,0,1,1
1653,1190,0,2.0,1,0,0,40,0.2,93,5,1,1399,1646,3610,13,7,9,0,0,1,3
559,1191,0,2.4,1,2,0,13,0.9,169,1,7,179,1813,1028,14,6,8,1,1,1,0


In [54]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv", index = False)

In [55]:
bucket

'mobbucketsagemaker0210'

In [56]:
sk_prefix="sagemaker/mobile_price_classificaton/sklearncontainer"
trainpath=sess.upload_data(path='train-V-1.csv',bucket=bucket,key_prefix=sk_prefix)
testpath=sess.upload_data(path='test-V-1.csv',bucket=bucket,key_prefix=sk_prefix)

print(trainpath)
print(testpath)


s3://mobbucketsagemaker0210/sagemaker/mobile_price_classificaton/sklearncontainer/train-V-1.csv
s3://mobbucketsagemaker0210/sagemaker/mobile_price_classificaton/sklearncontainer/test-V-1.csv


In [57]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score
import sklearn
import joblib
import boto3
import pathlib 
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd


def model_fn(model_dir):
    clf=joblib.load(os.path.join(model_dir,"model.joblib"))


if __name__=="__main__":

    print("[Info] Extracting arguments")
    parser=argparse.ArgumentParser()

    parser.add_argument("--n_estimators",type=int,default=100)
    parser.add_argument("--random_state",type=int,default=0)

    parser.add_argument("--model-dir",type=str,default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train",type=str,default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test",type=str,default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file",type=str,default="train-V-1.csv")
    parser.add_argument("--model-dir",type=str,default="test-V-1.csv")

    args, _ = parser.parse_known_args()

    print("Sklearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    lable = features.pop(-1)

    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    Y_train = train_df[lable]
    Y_test = test_df[lable] 

    print('Column order: ')
    print(features)
    print()
    
    print("Lable column is: ", lable)
    print()

    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(Y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(Y_test.shape)
    print()


    print("Training Random Forest Model ....")
    print()
    model=RandomForestClassifier(n_estimators=args.n_estimators,random_state=args.random_state,verbose=2,n_jobs=1)

    model.fit(X_train,Y_train)

    print()

    model_path=os.path.join(args.model_dir,"model.joblib")
    joblib.dump(model,model_path)

    print("Model saved at" + model_path)


    Y_pred_test= model.predict(X_test)
    test_acc = accuracy_score(Y_test,Y_pred_test)
    test_rep = classification_report(Y_test,Y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA")
    print("Total Rows are: ", X_test.shape[0])
    print("[TESTING] Model Accuracy is: ", test_acc)
    print("[TESTING] Testing Report: ")
    print(test_rep)
    


Overwriting script.py


In [58]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION= "0.23-1"

sklearn_estimatore=SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::879381254972:role/sagemakeraccess",
    isinstance_count= 1,
    isinstance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name = "RF-custom-sklearn",
    hyperparameters={
        "n_estimators":100,
        "random_state": 0,
    },
    use_spot_instance = True,
    max_wait=7200,
    max_run=3600

)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2                                                                                             │
│    3 FRAMEWORK_VERSION= "0.23-1"                                                                 │
│    4                                                                                             │
│ ❱  5 sklearn_estimatore=SKLearn(                                                                 │
│    6 │   entry_point="script.py",                                                                │
│    7 │   role="arn:aws:iam::879381254972:role/sagemakeraccess",                                  │
│    8 │   isinstance_count= 1,                                                                    │
│                                                                                                  │
│ d:\MLOps\awssagemaker\venv\lib\site-packages\sagemaker\sklearn\estimator.py:152 in __init__      │
│                                                                                                  │
│   149 │   │   │   if instance_count != 1:                                                        │
│   150 │   │   │   │   raise AttributeError(instance_cnt_err_msg)                                 │
│   151 │   │                                                                                      │
│ ❱ 152 │   │   super(SKLearn, self).__init__(                                                     │
│   153 │   │   │   entry_point,                                                                   │
│   154 │   │   │   source_dir,                                                                    │
│   155 │   │   │   hyperparameters,                                                               │
│                                                                                                  │
│ d:\MLOps\awssagemaker\venv\lib\site-packages\sagemaker\estimator.py:3571 in __init__             │
│                                                                                                  │
│   3568 │   │   │   You can find additional parameters for initializing this class at             │
│   3569 │   │   │   :class:`~sagemaker.estimator.EstimatorBase`.                                  │
│   3570 │   │   """                                                                               │
│ ❱ 3571 │   │   super(Framework, self).__init__(enable_network_isolation=enable_network_isolatio  │
│   3572 │   │   image_uri = renamed_kwargs("image_name", "image_uri", image_uri, kwargs)          │
│   3573 │   │                                                                                     │
│   3574 │   │   validate_source_code_input_against_pipeline_variables(                            │
│                                                                                                  │
│ d:\MLOps\awssagemaker\venv\lib\site-packages\sagemaker\estimator.py:630 in __init__              │
│                                                                                                  │
│    627 │   │   │   │   │   "https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode  │
│    628 │   │   │   │   )                                                                         │
│    629 │   │   else:                                                                             │
│ ❱  630 │   │   │   self.sagemaker_session = sagemaker_session or Session()                       │
│    631 │   │                                                                                     │
│    632 │   │   tags = format_tags(tags)                                                          │
│    633 │   │   self.tags = (                               